In [14]:
import pandas as pd
import ast
import numpy as np

In [8]:
movies = pd.read_csv('movies_clean.csv', parse_dates=['release_date'])
# Will need to re-download credits.csv from class (https://www.udemy.com/course/python-data-science-with-pandas-master-advanced-projects/learn/lecture/18644626#overview)
credits = pd.read_csv('credits.csv')

In [12]:
# Drop duplicate rows in credits DF
credits.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [16]:
# Change cast json string into Json Dict
credits['cast'] = credits['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [18]:
# View one movies cast as DF
pd.DataFrame(credits['cast'][0])

,cast_id,character,credit_id,gender,id,name,order,profile_path
0,14,Woody (voice),52fe4284c3a36847f8024f95,2,31,Tom Hanks,0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,15,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2,12898,Tim Allen,1,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,16,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2,7167,Don Rickles,2,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,17,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2,12899,Jim Varney,3,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,18,Rex (voice),52fe4284c3a36847f8024fa5,2,12900,Wallace Shawn,4,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg
5,19,Hamm (voice),52fe4284c3a36847f8024fa9,2,7907,John Ratzenberger,5,/yGechiKWL6TJDfVE2KPSJYqdMsY.jpg
6,20,Bo Peep (voice),52fe4284c3a36847f8024fad,1,8873,Annie Potts,6,/eryXT84RL41jHSJcMy4kS3u9y6w.jpg
7,26,Andy (voice),52fe4284c3a36847f8024fc1,0,1116442,John Morris,7,/vYGyvK4LzeaUCoNSHtsuqJUY15M.jpg
8,22,Sid (voice),52fe4284c3a36847f8024fb1,2,12901,Erik von Detten,8,/twnF1ZaJ1FUNUuo6xLXwcxjayBE.jpg
9,23,Mrs. Davis (voice),52fe4284c3a36847f8024fb5,1,12133,Laurie Metcalf,9,/unMMIT60eoBM2sN2nyR7EZ2BvvD.jpg


In [20]:
# Pull number of cast members
credits['castSize'] = credits['cast'].apply(lambda x: len(x))
credits['castSize'].value_counts(dropna=False).head(10)

10    2802
8     2766
7     2750
6     2691
5     2683
9     2579
4     2472
15    2457
0     2414
11    2304
Name: castSize, dtype: int64

In [36]:
# Create Actor string
def pipify(x, key):
    cast = []
    if not isinstance(x, list):
        print(1)
        return np.nan
    if len(x) == 0:
        print(2)
        return np.nan
    for i in x:
        cast.append(i[key])
    return "|".join(cast)

credits['cast'] = credits['cast'].head(10).apply(lambda x: pipify(x, 'name'))
credits['cast'].replace('', np.nan, inplace=True)

In [44]:
# Turn string literal crew column into json dict
credits['crew'] = credits['crew'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else np.nan)

In [46]:
# Get crew size from json dict of crew
credits['crew_size'] = credits['crew'].apply(lambda x: len(x))
credits['crew_size'].value_counts(dropna=False).head(10)

2     6298
1     5065
3     5014
4     3098
5     2270
7     1990
10    1948
8     1885
6     1885
9     1702
Name: crew_size, dtype: int64

In [48]:
# Get Director
def getJob(x, job):
    for i in x:
        if i['job'] == job:
            return i['name']
    return np.nan
        
credits['director'] = credits['crew'].apply(lambda x: getJob(x, 'Director'))

In [53]:
# Left join data into one df
df = movies.merge(credits, how='left', left_on='id', right_on='id')

In [54]:
df.drop(columns = 'crew', inplace=True)

In [55]:
df.to_csv('movies_complete.csv', index=False)